# Module 4.2: Logistic Regression - 邏輯回歸

## 學習目標

完成這個 notebook 後，你將能夠：

1. 理解 Sigmoid 函數的性質與意義
2. 理解 Binary Cross-Entropy 損失函數
3. 推導並實作 Logistic Regression 的梯度
4. 使用數值梯度檢查驗證實作
5. 視覺化決策邊界

## 背景知識

Logistic Regression 是最基礎的**分類**模型。雖然名字有 "Regression"，但它用於二分類問題。

### 從線性回歸到邏輯回歸

線性回歸輸出連續值，但分類問題需要輸出類別標籤。我們需要：

1. 將線性組合 $z = w^T x + b$ 映射到 $[0, 1]$ 區間
2. 解釋為 "屬於類別 1 的機率"

**Sigmoid 函數** 完成這個映射。

---

## 參考資源

- Bishop PRML Ch. 4: Linear Models for Classification

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 設定 matplotlib
plt.rcParams['figure.figsize'] = [10, 6]

# 設定隨機種子
np.random.seed(42)

print("環境設定完成！")
print(f"NumPy 版本: {np.__version__}")

---

## Part 1: Sigmoid 函數

### 定義

$$\sigma(z) = \frac{1}{1 + e^{-z}}$$

### 性質

1. **值域**：$\sigma(z) \in (0, 1)$
2. **單調遞增**：$z$ 越大，$\sigma(z)$ 越大
3. **對稱性**：$\sigma(-z) = 1 - \sigma(z)$
4. **導數**：$\sigma'(z) = \sigma(z)(1 - \sigma(z))$

### 數值穩定性

直接計算 $e^{-z}$ 在 $z$ 很負時會溢出。使用以下技巧：

$$\sigma(z) = \begin{cases} 
\frac{1}{1 + e^{-z}} & \text{if } z \geq 0 \\
\frac{e^z}{1 + e^z} & \text{if } z < 0
\end{cases}$$

In [ ]:
def sigmoid_naive(z):
    """
    Sigmoid 函數（簡單版，可能有數值問題）
    """
    return 1 / (1 + np.exp(-z))


def sigmoid(z):
    """
    Sigmoid 函數（數值穩定版）
    
    Parameters
    ----------
    z : np.ndarray or float
        輸入值
    
    Returns
    -------
    np.ndarray or float
        Sigmoid 輸出，範圍 (0, 1)
    """
    z = np.asarray(z, dtype=np.float64)
    result = np.zeros_like(z)
    
    # z >= 0: 用標準公式
    pos_mask = z >= 0
    result[pos_mask] = 1 / (1 + np.exp(-z[pos_mask]))
    
    # z < 0: 用等價形式避免 exp 溢出
    neg_mask = z < 0
    exp_z = np.exp(z[neg_mask])
    result[neg_mask] = exp_z / (1 + exp_z)
    
    return result


# 測試 Sigmoid
print("=== 測試 Sigmoid 函數 ===")

# 基本測試
test_values = [-10, -5, -1, 0, 1, 5, 10]
print(f"{'z':>6} | {'σ(z)':>10}")
print("-" * 20)
for z in test_values:
    print(f"{z:>6} | {sigmoid(z):>10.6f}")

# 驗證性質
print("\n驗證性質：")
print(f"σ(0) = {sigmoid(0):.6f} (應該是 0.5)")
print(f"σ(-5) + σ(5) = {sigmoid(-5) + sigmoid(5):.6f} (應該是 1.0)")

# 導數驗證
z = 2.0
s = sigmoid(z)
dsigmoid_analytic = s * (1 - s)

# 數值導數
eps = 1e-5
dsigmoid_numeric = (sigmoid(z + eps) - sigmoid(z - eps)) / (2 * eps)

print(f"\n導數驗證 (z={z}):")
print(f"解析導數: {dsigmoid_analytic:.8f}")
print(f"數值導數: {dsigmoid_numeric:.8f}")

In [ ]:
# 視覺化 Sigmoid
z = np.linspace(-10, 10, 200)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Sigmoid 函數
axes[0].plot(z, sigmoid(z), 'b-', linewidth=2)
axes[0].axhline(y=0.5, color='r', linestyle='--', alpha=0.5)
axes[0].axvline(x=0, color='r', linestyle='--', alpha=0.5)
axes[0].set_xlabel('z')
axes[0].set_ylabel('σ(z)')
axes[0].set_title('Sigmoid 函數')
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim(-0.1, 1.1)

# Sigmoid 導數
s = sigmoid(z)
ds = s * (1 - s)
axes[1].plot(z, ds, 'g-', linewidth=2)
axes[1].set_xlabel('z')
axes[1].set_ylabel("σ'(z)")
axes[1].set_title('Sigmoid 導數: σ(z)(1-σ(z))')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("觀察：導數在 z=0 時最大 (0.25)，在兩端趨近於 0")
print("這意味著梯度在預測很確定時會變小（梯度消失問題）")

---

## Part 2: Binary Cross-Entropy Loss

### 為什麼不用 MSE？

如果用 MSE：$L = (\sigma(z) - y)^2$，損失函數會是**非凸**的，有多個局部最小值。

### Cross-Entropy 定義

對單一樣本：

$$L(y, \hat{y}) = -[y \log(\hat{y}) + (1-y) \log(1-\hat{y})]$$

其中 $\hat{y} = \sigma(w^T x + b) \in (0, 1)$，$y \in \{0, 1\}$

### 直覺理解

- 當 $y = 1$ 時：$L = -\log(\hat{y})$
  - $\hat{y}$ 越接近 1，loss 越小
  - $\hat{y}$ 接近 0 時，$-\log(\hat{y}) \to \infty$

- 當 $y = 0$ 時：$L = -\log(1 - \hat{y})$
  - $\hat{y}$ 越接近 0，loss 越小

### 批次形式

$$L = -\frac{1}{N} \sum_{i=1}^N [y_i \log(\hat{y}_i) + (1-y_i) \log(1-\hat{y}_i)]$$

In [ ]:
def binary_cross_entropy(y_true, y_pred, eps=1e-15):
    """
    計算 Binary Cross-Entropy Loss
    
    Parameters
    ----------
    y_true : np.ndarray, shape (N,)
        真實標籤，值為 0 或 1
    y_pred : np.ndarray, shape (N,)
        預測機率，值在 (0, 1)
    eps : float
        避免 log(0) 的小常數
    
    Returns
    -------
    float
        平均 BCE loss
    """
    # Clip 預測值避免 log(0)
    y_pred = np.clip(y_pred, eps, 1 - eps)
    
    # BCE
    loss = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    
    return loss


# 視覺化 BCE
print("=== 視覺化 Binary Cross-Entropy ===")

y_pred_range = np.linspace(0.001, 0.999, 100)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# y = 1 的情況
loss_y1 = -np.log(y_pred_range)
axes[0].plot(y_pred_range, loss_y1, 'b-', linewidth=2, label='BCE when y=1')
axes[0].set_xlabel('Predicted probability ŷ')
axes[0].set_ylabel('Loss')
axes[0].set_title('BCE Loss when y=1: -log(ŷ)')
axes[0].set_ylim(0, 5)
axes[0].grid(True, alpha=0.3)
axes[0].axvline(x=0.5, color='r', linestyle='--', alpha=0.5)

# y = 0 的情況
loss_y0 = -np.log(1 - y_pred_range)
axes[1].plot(y_pred_range, loss_y0, 'r-', linewidth=2, label='BCE when y=0')
axes[1].set_xlabel('Predicted probability ŷ')
axes[1].set_ylabel('Loss')
axes[1].set_title('BCE Loss when y=0: -log(1-ŷ)')
axes[1].set_ylim(0, 5)
axes[1].grid(True, alpha=0.3)
axes[1].axvline(x=0.5, color='r', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

# 測試
print("\n測試 BCE：")
print(f"y=1, ŷ=0.9: Loss = {binary_cross_entropy(np.array([1]), np.array([0.9])):.4f}")
print(f"y=1, ŷ=0.5: Loss = {binary_cross_entropy(np.array([1]), np.array([0.5])):.4f}")
print(f"y=1, ŷ=0.1: Loss = {binary_cross_entropy(np.array([1]), np.array([0.1])):.4f}")

---

## Part 3: 梯度推導

### 目標

計算 $\frac{\partial L}{\partial w}$ 和 $\frac{\partial L}{\partial b}$

### 推導過程

令 $z = w^T x + b$，$\hat{y} = \sigma(z)$

$$L = -[y \log(\hat{y}) + (1-y) \log(1-\hat{y})]$$

使用鏈式法則：

$$\frac{\partial L}{\partial w} = \frac{\partial L}{\partial \hat{y}} \cdot \frac{\partial \hat{y}}{\partial z} \cdot \frac{\partial z}{\partial w}$$

**Step 1**: $\frac{\partial L}{\partial \hat{y}}$

$$\frac{\partial L}{\partial \hat{y}} = -\frac{y}{\hat{y}} + \frac{1-y}{1-\hat{y}} = \frac{\hat{y} - y}{\hat{y}(1-\hat{y})}$$

**Step 2**: $\frac{\partial \hat{y}}{\partial z} = \sigma(z)(1-\sigma(z)) = \hat{y}(1-\hat{y})$

**Step 3**: $\frac{\partial z}{\partial w} = x$

### 漂亮的結果

$$\frac{\partial L}{\partial w} = \frac{\hat{y} - y}{\hat{y}(1-\hat{y})} \cdot \hat{y}(1-\hat{y}) \cdot x = (\hat{y} - y) \cdot x$$

$$\frac{\partial L}{\partial b} = \hat{y} - y$$

**批次形式**：

$$\frac{\partial L}{\partial w} = \frac{1}{N} X^T (\hat{y} - y)$$
$$\frac{\partial L}{\partial b} = \frac{1}{N} \sum_i (\hat{y}_i - y_i)$$

這和線性回歸的梯度形式一樣！

In [ ]:
def compute_logistic_gradients(X, y, w, b):
    """
    計算 Logistic Regression 的梯度
    
    Parameters
    ----------
    X : np.ndarray, shape (N, D)
        特徵矩陣
    y : np.ndarray, shape (N,)
        真實標籤（0 或 1）
    w : np.ndarray, shape (D,)
        權重
    b : float
        偏置
    
    Returns
    -------
    dw : np.ndarray, shape (D,)
        ∂L/∂w
    db : float
        ∂L/∂b
    """
    N = X.shape[0]
    
    # 前向傳播
    z = X @ w + b
    y_pred = sigmoid(z)
    
    # 梯度（漂亮的結果！）
    error = y_pred - y  # shape (N,)
    
    dw = (1 / N) * (X.T @ error)
    db = (1 / N) * np.sum(error)
    
    return dw, db


# 數值梯度檢查
def numerical_gradient(f, x, eps=1e-5):
    """計算數值梯度"""
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        old_value = x[idx]
        
        x[idx] = old_value + eps
        fx_plus = f(x)
        
        x[idx] = old_value - eps
        fx_minus = f(x)
        
        x[idx] = old_value
        grad[idx] = (fx_plus - fx_minus) / (2 * eps)
        
        it.iternext()
    
    return grad


def gradient_check_logistic(X, y, w, b, eps=1e-5):
    """
    檢查 Logistic Regression 的梯度
    """
    # 解析梯度
    dw_analytic, db_analytic = compute_logistic_gradients(X, y, w, b)
    
    # 數值梯度 for w
    def loss_w(w_test):
        z = X @ w_test + b
        y_pred = sigmoid(z)
        return binary_cross_entropy(y, y_pred)
    
    dw_numeric = numerical_gradient(loss_w, w.copy(), eps)
    
    # 數值梯度 for b
    def loss_b(b_test):
        z = X @ w + b_test[0]
        y_pred = sigmoid(z)
        return binary_cross_entropy(y, y_pred)
    
    db_numeric = numerical_gradient(loss_b, np.array([b]), eps)[0]
    
    # 比較
    print("=== 梯度檢查 ===")
    print(f"dw 解析梯度: {dw_analytic}")
    print(f"dw 數值梯度: {dw_numeric}")
    rel_err_w = np.linalg.norm(dw_analytic - dw_numeric) / (np.linalg.norm(dw_analytic) + np.linalg.norm(dw_numeric) + 1e-8)
    print(f"dw 相對誤差: {rel_err_w:.2e}")
    print()
    print(f"db 解析梯度: {db_analytic:.8f}")
    print(f"db 數值梯度: {db_numeric:.8f}")
    rel_err_b = abs(db_analytic - db_numeric) / (abs(db_analytic) + abs(db_numeric) + 1e-8)
    print(f"db 相對誤差: {rel_err_b:.2e}")
    
    passed = rel_err_w < 1e-4 and rel_err_b < 1e-4
    print(f"\n梯度檢查通過: {passed}")
    return passed


# 生成測試資料
np.random.seed(42)
X_test = np.random.randn(50, 3)
y_test = (np.random.rand(50) > 0.5).astype(float)
w_test = np.random.randn(3)
b_test = np.random.randn()

gradient_check_logistic(X_test, y_test, w_test, b_test)

---

## Part 4: 完整的 Logistic Regression 實作

In [ ]:
class LogisticRegression:
    """
    Logistic Regression 二分類模型
    
    使用梯度下降優化 Binary Cross-Entropy Loss
    
    Parameters
    ----------
    regularization : float
        L2 正則化強度
    
    Attributes
    ----------
    w : np.ndarray
        權重向量
    b : float
        偏置
    history : dict
        訓練歷史
    """
    
    def __init__(self, regularization=0.0):
        self.reg = regularization
        self.w = None
        self.b = None
        self.history = None
    
    def _sigmoid(self, z):
        """數值穩定的 Sigmoid"""
        return sigmoid(z)
    
    def fit(self, X, y, lr=0.1, n_iter=1000, verbose=True):
        """
        使用梯度下降訓練模型
        
        Parameters
        ----------
        X : np.ndarray, shape (N, D)
            特徵矩陣
        y : np.ndarray, shape (N,)
            標籤（0 或 1）
        lr : float
            學習率
        n_iter : int
            迭代次數
        verbose : bool
            是否輸出訓練過程
        """
        N, D = X.shape
        
        # 初始化參數
        self.w = np.zeros(D)
        self.b = 0.0
        self.history = {'loss': [], 'accuracy': []}
        
        for i in range(n_iter):
            # 前向傳播
            z = X @ self.w + self.b
            y_pred = self._sigmoid(z)
            
            # 計算 loss（含正則化）
            loss = binary_cross_entropy(y, y_pred) + 0.5 * self.reg * np.sum(self.w ** 2)
            
            # 計算準確率
            predictions = (y_pred >= 0.5).astype(float)
            accuracy = np.mean(predictions == y)
            
            # 記錄
            self.history['loss'].append(loss)
            self.history['accuracy'].append(accuracy)
            
            # 計算梯度
            error = y_pred - y
            dw = (1 / N) * (X.T @ error) + self.reg * self.w
            db = (1 / N) * np.sum(error)
            
            # 更新參數
            self.w = self.w - lr * dw
            self.b = self.b - lr * db
            
            if verbose and (i + 1) % (n_iter // 10) == 0:
                print(f"Iter {i+1:4d}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}")
        
        return self
    
    def predict_proba(self, X):
        """
        預測屬於類別 1 的機率
        
        Returns
        -------
        np.ndarray, shape (N,)
            P(y=1|x)
        """
        if self.w is None:
            raise ValueError("Model not fitted. Call fit() first.")
        z = X @ self.w + self.b
        return self._sigmoid(z)
    
    def predict(self, X, threshold=0.5):
        """
        預測類別標籤
        
        Returns
        -------
        np.ndarray, shape (N,)
            預測標籤（0 或 1）
        """
        proba = self.predict_proba(X)
        return (proba >= threshold).astype(int)
    
    def score(self, X, y):
        """
        計算準確率
        """
        predictions = self.predict(X)
        return np.mean(predictions == y)


# 生成二分類資料
def generate_binary_classification_data(n_samples=200, seed=42):
    """
    生成二分類測試資料
    
    兩個高斯分布的混合
    """
    np.random.seed(seed)
    
    n_per_class = n_samples // 2
    
    # 類別 0
    X0 = np.random.randn(n_per_class, 2) * 0.8 + np.array([-1, -1])
    y0 = np.zeros(n_per_class)
    
    # 類別 1
    X1 = np.random.randn(n_per_class, 2) * 0.8 + np.array([1, 1])
    y1 = np.ones(n_per_class)
    
    # 合併
    X = np.vstack([X0, X1])
    y = np.concatenate([y0, y1])
    
    # 打亂
    idx = np.random.permutation(n_samples)
    return X[idx], y[idx]


# 測試
print("=== 測試 Logistic Regression ===")

X, y = generate_binary_classification_data(n_samples=200)

print(f"資料大小: {X.shape}")
print(f"類別分布: {np.bincount(y.astype(int))}")

# 訓練模型
model = LogisticRegression(regularization=0.01)
model.fit(X, y, lr=0.5, n_iter=500, verbose=True)

print(f"\n最終準確率: {model.score(X, y):.4f}")
print(f"學習到的參數: w = {model.w}, b = {model.b:.4f}")

In [ ]:
# 視覺化結果

def plot_decision_boundary(model, X, y, title="Decision Boundary"):
    """
    繪製決策邊界
    """
    # 建立網格
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                         np.linspace(y_min, y_max, 200))
    
    # 預測網格上每個點
    Z = model.predict_proba(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # 繪圖
    plt.figure(figsize=(10, 8))
    
    # 機率等高線
    plt.contourf(xx, yy, Z, levels=np.linspace(0, 1, 11), cmap='RdYlBu_r', alpha=0.6)
    plt.colorbar(label='P(y=1|x)')
    
    # 決策邊界（P=0.5）
    plt.contour(xx, yy, Z, levels=[0.5], colors='black', linewidths=2)
    
    # 資料點
    plt.scatter(X[y==0, 0], X[y==0, 1], c='blue', marker='o', s=50, label='Class 0', edgecolors='k')
    plt.scatter(X[y==1, 0], X[y==1, 1], c='red', marker='s', s=50, label='Class 1', edgecolors='k')
    
    plt.xlabel('$x_1$')
    plt.ylabel('$x_2$')
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    return plt.gcf()


# 決策邊界
plot_decision_boundary(model, X, y, title='Logistic Regression 決策邊界')
plt.show()

# 訓練曲線
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(model.history['loss'])
axes[0].set_xlabel('Iteration')
axes[0].set_ylabel('Loss (BCE + L2)')
axes[0].set_title('訓練 Loss 曲線')
axes[0].grid(True, alpha=0.3)

axes[1].plot(model.history['accuracy'])
axes[1].set_xlabel('Iteration')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('訓練準確率曲線')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Part 5: 決策邊界的幾何意義

### 決策邊界方程

決策邊界是 $P(y=1|x) = 0.5$ 的點集，即：

$$\sigma(w^T x + b) = 0.5 \Rightarrow w^T x + b = 0$$

這是一個**超平面**（2D 中是直線）。

### 法向量

$w$ 是超平面的法向量，指向類別 1 的方向。

In [ ]:
# 視覺化決策邊界的幾何

plt.figure(figsize=(10, 8))

# 資料點
plt.scatter(X[y==0, 0], X[y==0, 1], c='blue', marker='o', s=50, label='Class 0', edgecolors='k')
plt.scatter(X[y==1, 0], X[y==1, 1], c='red', marker='s', s=50, label='Class 1', edgecolors='k')

# 決策邊界直線
# w1*x1 + w2*x2 + b = 0 => x2 = -(w1*x1 + b) / w2
w1, w2 = model.w
b = model.b

x1_range = np.linspace(-4, 4, 100)
x2_boundary = -(w1 * x1_range + b) / w2

plt.plot(x1_range, x2_boundary, 'k-', linewidth=2, label='Decision boundary')

# 法向量 w（從原點指向）
origin = np.array([0, 0])
w_normalized = model.w / np.linalg.norm(model.w) * 2  # 縮放顯示
plt.arrow(origin[0], origin[1], w_normalized[0], w_normalized[1], 
          head_width=0.15, head_length=0.1, fc='green', ec='green', linewidth=2)
plt.text(w_normalized[0] + 0.2, w_normalized[1] + 0.2, 'w (normal)', fontsize=12, color='green')

plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.title('決策邊界的幾何意義')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.xlim(-4, 4)
plt.ylim(-4, 4)
plt.show()

print(f"\n決策邊界方程: {w1:.4f}*x1 + {w2:.4f}*x2 + {b:.4f} = 0")

---

## 練習題

### 練習 1：非線性可分資料

**任務**：在非線性可分資料上測試 Logistic Regression

**觀察**：線性模型的局限性

In [ ]:
# 練習 1 解答

def generate_xor_data(n_samples=200, noise=0.1, seed=42):
    """
    生成 XOR 模式的資料（非線性可分）
    """
    np.random.seed(seed)
    
    n_per_class = n_samples // 4
    
    # 四個象限
    X1 = np.random.randn(n_per_class, 2) * 0.5 + np.array([1, 1])
    X2 = np.random.randn(n_per_class, 2) * 0.5 + np.array([-1, -1])
    X3 = np.random.randn(n_per_class, 2) * 0.5 + np.array([1, -1])
    X4 = np.random.randn(n_per_class, 2) * 0.5 + np.array([-1, 1])
    
    X = np.vstack([X1, X2, X3, X4])
    y = np.array([1] * n_per_class + [1] * n_per_class + 
                 [0] * n_per_class + [0] * n_per_class)
    
    idx = np.random.permutation(len(y))
    return X[idx], y[idx].astype(float)


# 生成 XOR 資料
X_xor, y_xor = generate_xor_data(n_samples=200)

# 訓練線性 Logistic Regression
model_linear = LogisticRegression(regularization=0.01)
model_linear.fit(X_xor, y_xor, lr=0.5, n_iter=500, verbose=False)

print(f"線性模型在 XOR 資料上的準確率: {model_linear.score(X_xor, y_xor):.4f}")
print("(接近 0.5 表示模型無法學習)")

# 視覺化
plot_decision_boundary(model_linear, X_xor, y_xor, 
                       title='XOR 資料 - 線性 Logistic Regression（失敗）')
plt.show()

### 練習 2：特徵工程解決非線性問題

**任務**：通過添加非線性特徵來解決 XOR 問題

**提示**：加入 $x_1 \cdot x_2$ 特徵

In [ ]:
# 練習 2 解答

def add_interaction_feature(X):
    """
    添加交互特徵 x1 * x2
    """
    x1_x2 = (X[:, 0] * X[:, 1]).reshape(-1, 1)
    return np.hstack([X, x1_x2])


# 添加交互特徵
X_xor_extended = add_interaction_feature(X_xor)
print(f"原始特徵維度: {X_xor.shape[1]}")
print(f"擴展後特徵維度: {X_xor_extended.shape[1]}")

# 重新訓練
model_extended = LogisticRegression(regularization=0.01)
model_extended.fit(X_xor_extended, y_xor, lr=0.5, n_iter=500, verbose=False)

print(f"\n加入交互特徵後的準確率: {model_extended.score(X_xor_extended, y_xor):.4f}")

# 視覺化決策邊界
# 這需要在原始 2D 空間中繪製
x_min, x_max = X_xor[:, 0].min() - 1, X_xor[:, 0].max() + 1
y_min, y_max = X_xor[:, 1].min() - 1, X_xor[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                     np.linspace(y_min, y_max, 200))

# 加入交互特徵後預測
grid_points = np.c_[xx.ravel(), yy.ravel()]
grid_extended = add_interaction_feature(grid_points)
Z = model_extended.predict_proba(grid_extended)
Z = Z.reshape(xx.shape)

plt.figure(figsize=(10, 8))
plt.contourf(xx, yy, Z, levels=np.linspace(0, 1, 11), cmap='RdYlBu_r', alpha=0.6)
plt.colorbar(label='P(y=1|x)')
plt.contour(xx, yy, Z, levels=[0.5], colors='black', linewidths=2)
plt.scatter(X_xor[y_xor==0, 0], X_xor[y_xor==0, 1], c='blue', marker='o', s=50, label='Class 0', edgecolors='k')
plt.scatter(X_xor[y_xor==1, 0], X_xor[y_xor==1, 1], c='red', marker='s', s=50, label='Class 1', edgecolors='k')
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.title('XOR 資料 - 加入 $x_1 \cdot x_2$ 特徵後的決策邊界')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"\n學習到的參數: w = {model_extended.w}, b = {model_extended.b:.4f}")
print("決策邊界: w1*x1 + w2*x2 + w3*(x1*x2) + b = 0")
print("這是一個雙曲線！")

### 練習 3：多項式特徵

**任務**：用多項式特徵處理圓形分佈的資料

In [ ]:
# 練習 3 解答

def generate_circle_data(n_samples=200, seed=42):
    """
    生成圓形分佈資料
    """
    np.random.seed(seed)
    
    n_per_class = n_samples // 2
    
    # 內圈（類別 0）
    r_inner = np.random.uniform(0, 1, n_per_class)
    theta_inner = np.random.uniform(0, 2*np.pi, n_per_class)
    X_inner = np.column_stack([r_inner * np.cos(theta_inner), 
                               r_inner * np.sin(theta_inner)])
    
    # 外圈（類別 1）
    r_outer = np.random.uniform(2, 3, n_per_class)
    theta_outer = np.random.uniform(0, 2*np.pi, n_per_class)
    X_outer = np.column_stack([r_outer * np.cos(theta_outer), 
                               r_outer * np.sin(theta_outer)])
    
    X = np.vstack([X_inner, X_outer])
    y = np.array([0] * n_per_class + [1] * n_per_class).astype(float)
    
    idx = np.random.permutation(n_samples)
    return X[idx], y[idx]


def polynomial_features_2d(X, degree=2):
    """
    生成 2D 多項式特徵
    
    degree=2: [1, x1, x2, x1^2, x1*x2, x2^2]
    """
    x1, x2 = X[:, 0], X[:, 1]
    features = [np.ones(len(x1))]  # bias（會被模型加，這裡先不加）
    
    for d in range(1, degree + 1):
        for i in range(d + 1):
            features.append((x1 ** (d - i)) * (x2 ** i))
    
    return np.column_stack(features[1:])  # 去掉常數項


# 生成圓形資料
X_circle, y_circle = generate_circle_data(n_samples=200)

# 線性模型
model_linear = LogisticRegression(regularization=0.01)
model_linear.fit(X_circle, y_circle, lr=0.5, n_iter=500, verbose=False)
print(f"線性模型準確率: {model_linear.score(X_circle, y_circle):.4f}")

# 多項式特徵模型
X_circle_poly = polynomial_features_2d(X_circle, degree=2)
print(f"多項式特徵維度: {X_circle_poly.shape[1]}")

model_poly = LogisticRegression(regularization=0.01)
model_poly.fit(X_circle_poly, y_circle, lr=0.1, n_iter=1000, verbose=False)
print(f"多項式模型準確率: {model_poly.score(X_circle_poly, y_circle):.4f}")

# 視覺化
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 線性模型
x_min, x_max = X_circle[:, 0].min() - 0.5, X_circle[:, 0].max() + 0.5
y_min, y_max = X_circle[:, 1].min() - 0.5, X_circle[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                     np.linspace(y_min, y_max, 200))

Z_linear = model_linear.predict_proba(np.c_[xx.ravel(), yy.ravel()])
Z_linear = Z_linear.reshape(xx.shape)

axes[0].contourf(xx, yy, Z_linear, levels=np.linspace(0, 1, 11), cmap='RdYlBu_r', alpha=0.6)
axes[0].contour(xx, yy, Z_linear, levels=[0.5], colors='black', linewidths=2)
axes[0].scatter(X_circle[y_circle==0, 0], X_circle[y_circle==0, 1], c='blue', s=30, label='Class 0')
axes[0].scatter(X_circle[y_circle==1, 0], X_circle[y_circle==1, 1], c='red', s=30, label='Class 1')
axes[0].set_title(f'線性模型 (Acc: {model_linear.score(X_circle, y_circle):.2f})')
axes[0].legend()

# 多項式模型
grid_points = np.c_[xx.ravel(), yy.ravel()]
grid_poly = polynomial_features_2d(grid_points, degree=2)
Z_poly = model_poly.predict_proba(grid_poly)
Z_poly = Z_poly.reshape(xx.shape)

axes[1].contourf(xx, yy, Z_poly, levels=np.linspace(0, 1, 11), cmap='RdYlBu_r', alpha=0.6)
axes[1].contour(xx, yy, Z_poly, levels=[0.5], colors='black', linewidths=2)
axes[1].scatter(X_circle[y_circle==0, 0], X_circle[y_circle==0, 1], c='blue', s=30, label='Class 0')
axes[1].scatter(X_circle[y_circle==1, 0], X_circle[y_circle==1, 1], c='red', s=30, label='Class 1')
axes[1].set_title(f'二次多項式模型 (Acc: {model_poly.score(X_circle_poly, y_circle):.2f})')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n觀察：加入 x1², x1*x2, x2² 特徵後，可以學習圓形邊界")

---

## 總結

### 本 Notebook 涵蓋的內容

1. **Sigmoid 函數**：
   - $\sigma(z) = \frac{1}{1 + e^{-z}}$
   - 將任意值映射到 $(0, 1)$
   - 數值穩定實作

2. **Binary Cross-Entropy**：
   - $L = -[y\log(\hat{y}) + (1-y)\log(1-\hat{y})]$
   - 比 MSE 更適合分類

3. **梯度推導**：
   - $\frac{\partial L}{\partial w} = \frac{1}{N} X^T (\hat{y} - y)$
   - 形式和線性回歸一樣漂亮！

4. **決策邊界**：
   - $w^T x + b = 0$ 定義超平面
   - 線性模型只能學習線性邊界

5. **特徵工程**：
   - 添加非線性特徵可以處理非線性可分資料
   - 這是「核方法」的前身

### 關鍵要點

1. **梯度檢查**：永遠驗證你的梯度實作
2. **數值穩定性**：處理 exp 和 log 時要小心
3. **線性模型的局限**：無法處理非線性可分資料

### 下一步

在下一個 notebook 中，我們將學習 **Softmax Regression**，將二分類推廣到多分類問題。